## 전처리

결측치 제거하기

In [1]:
import pandas as pd

In [2]:
#원본파일(data.csv) 읽어오기
file_path = 'data.csv'
df = pd.read_csv(file_path, encoding='cp949')

#결측치 제거
df_cleaned = df.dropna()

#결측치 제거 후 파일 저장
cleaned_file_path = '1_data.csv'
df_cleaned.to_csv(cleaned_file_path, index=False, encoding='cp949')

평균, 표준편차
<hr>
평균, 표준편차 사용 이유
<br> : X 변수들은 시계열 데이터지만 Y 변수는 배정번호 별 데이터로 되어있기 때문에 X 변수들의 배정번호별 통계값(평균, 표준편차) 산출이 필요함

In [3]:
#결측치 제거 파일 읽어오기
file_path = '1_data.csv'
df2 = pd.read_csv(file_path, encoding='cp949')

# 배정번호 컬럼 (그룹화 기준이 되는 컬럼명)
assignment_column = '배정번호'

# 숫자형 컬럼만 선택
numeric_columns = df2.select_dtypes(include=['number']).columns.drop(assignment_column)
grouped = df2.groupby(assignment_column)[numeric_columns].agg(['mean', 'std']).reset_index()

# 컬럼명 정리
grouped.columns = ['_'.join(col).strip() if col[1] else col[0] for col in grouped.columns.values]

# 결과를 새로운 CSV 파일로 저장
output_file_path = '2_data.csv'
grouped.to_csv(output_file_path, index=False, encoding='cp949')

불량률 계산

In [4]:
file_path = '열처리_품질데이터.xlsx'
df = pd.read_excel(file_path)

# 불량 비율 계산
df['불량률'] = df['불량수량'] / df['총수량']

result_file_path = '1_열처리_품질데이터.xlsx'
df.to_excel(result_file_path, index=False)

불량률 평균

In [5]:
file_path = '1_열처리_품질데이터.xlsx'
df = pd.read_excel(file_path)
average_defect_rate = df['불량률'].mean()
print(f'불량률의 평균: {average_defect_rate}')

불량률의 평균: 0.00037013106122237856


불량률 배정번호에 맞게 넣어주기

In [8]:
xlsx_file_path = '1_열처리_품질데이터.xlsx'
csv_file_path = '2_data.csv'

df_excel = pd.read_excel(xlsx_file_path)
df_csv = pd.read_csv(csv_file_path, encoding='cp949')

# 위험, 안전 분류(0.0003 넘으면 위험, 아니면 안전)
df_excel['상태'] = df_excel['불량률'].apply(lambda x: '위험' if x >= 0.0003 else '안전')

# 필요한 열 선택
df_excel_status = df_excel[['배정번호', '상태']]

# 두 데이터프레임 병합(배정번호 기준)
df_merged = pd.merge(df_csv, df_excel_status, on='배정번호', how='left')

# '배정번호' 열 제거
df_merged = df_merged.drop(columns=['배정번호'])

#print(df_merged.columns)

# 열 제거
columns_to_drop = ['소입로 CP 모니터 값_mean', '소입로 CP 모니터 값_std']
df_merged = df_merged.drop(columns=columns_to_drop, axis=1)

result_file_path = '1_train.csv'
df_merged.to_csv(result_file_path, index=False, encoding='cp949')

Index(['건조 1존 OP_mean', '건조 1존 OP_std', '건조 2존 OP_mean', '건조 2존 OP_std',
       '건조로 온도 1 Zone_mean', '건조로 온도 1 Zone_std', '건조로 온도 2 Zone_mean',
       '건조로 온도 2 Zone_std', '세정기_mean', '세정기_std', '소입1존 OP_mean',
       '소입1존 OP_std', '소입2존 OP_mean', '소입2존 OP_std', '소입3존 OP_mean',
       '소입3존 OP_std', '소입4존 OP_mean', '소입4존 OP_std', '소입로 CP 값_mean',
       '소입로 CP 값_std', '소입로 CP 모니터 값_mean', '소입로 CP 모니터 값_std',
       '소입로 온도 1 Zone_mean', '소입로 온도 1 Zone_std', '소입로 온도 2 Zone_mean',
       '소입로 온도 2 Zone_std', '소입로 온도 3 Zone_mean', '소입로 온도 3 Zone_std',
       '소입로 온도 4 Zone_mean', '소입로 온도 4 Zone_std', '솔트 컨베이어 온도 1 Zone_mean',
       '솔트 컨베이어 온도 1 Zone_std', '솔트 컨베이어 온도 2 Zone_mean',
       '솔트 컨베이어 온도 2 Zone_std', '솔트조 온도 1 Zone_mean', '솔트조 온도 1 Zone_std',
       '솔트조 온도 2 Zone_mean', '솔트조 온도 2 Zone_std', '상태'],
      dtype='object')
